In [ ]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [ ]:
from os import getcwd, path, pardir
import pickle
from math import ceil

In [ ]:
run_experiments = True

Experiment AES

In [ ]:
if run_experiments:
    from Security.AES.experiment_aes import experiment_aes
    experiment_aes(path.join(getcwd(), "Security", "AES"))

encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_encrypt_charge.p"), "rb"))
decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_decrypt_charge.p"), "rb"))

Experiment AES Flash

In [ ]:
if run_experiments:
    from Security.AES_Flash.experiment_aes_flash import experiment_aes_flash
    experiment_aes_flash(path.join(getcwd(), "Security", "AES_Flash"))

aes_encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_encrypt_charge.p"), "rb"))
aes_decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_decrypt_charge.p"), "rb"))
flash_write_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_write_charge.p"), "rb"))
flash_read_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_read_charge.p"), "rb"))

Experiment TrustZone

In [ ]:
if run_experiments:
    from Security.TrustZone.experiment_trustzone import experiment_trustzone
    experiment_trustzone(path.join(getcwd(), "Security", "TrustZone"))

nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_store_charge.p"), "rb"))
nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_load_charge.p"), "rb"))

Experiment TrustZone Flash

In [ ]:
if run_experiments:
    from Security.TrustZone_Flash.experiment_trustzone_flash import experiment_trustzone_flash
    experiment_trustzone_flash(path.join(getcwd(), "Security", "TrustZone_Flash"))

flash_nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
flash_nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))

Make lookup function

In [ ]:
AES_BLOCK_SIZE = 16
def get_security_charge(security_type, number_of_bytes):
    if security_type is "AES":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return encrypt_charge[index] + decrypt_charge[index]
    elif security_type is "AES_Flash":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return aes_encrypt_charge[index] + flash_write_charge[index] + flash_read_charge[index] + aes_decrypt_charge[index]
    elif security_type is "TrustZone":
        return nsc_store_charge[number_of_bytes] + nsc_store_charge[number_of_bytes]
    elif security_type is "TrustZone_Flash":
        return flash_nsc_store_charge[number_of_bytes] + flash_nsc_load_charge[number_of_bytes]
    else:
        return 0 # No security, so 0 C

In [ ]:
security_types = ["None", "AES", "AES_Flash", "TrustZone", "TrustZone_Flash"]
# security_types = ["None", "AES_Flash", "TruseZone_Flash"]
application_names = ["Kalman", "AR", "Dijkstra", "Sorting 1024", "Bit Count", "fft"]

In [ ]:
# app_charge_scale = 1e-4
application_charges = np.array([0.0001, 0.0002, 0.0003, 0.00001984909516280825944492365031646130546505446545779705047607421875, 0.0004, 0.0005]) # TODO
# app_number_of_bytes_scale = 1000
application_number_of_bytes = np.array([234, 32, 876, 1544 + 2**10, 90, 95]) # TODO

In [ ]:
base_charge_per_app = dict(zip(application_names, application_charges))
number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

In [ ]:
def get_total_charge(application_name, security_type):
    print(application_name, security_type, f"base: {base_charge_per_app[application_name]}, sec: {get_security_charge(security_type, number_of_bytes_per_app[application_name])}")
    return base_charge_per_app[application_name] + get_security_charge(security_type, number_of_bytes_per_app[application_name])

# Bar chart

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# create plot
fig, ax = plt.subplots(figsize=(10,9))
index = np.arange(len(application_names))
bar_width = 0.18
opacity = 0.8

# ax.grid()

colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

rects = []
for i,security_type in enumerate(security_types):
    rects += ax.bar(index + i*bar_width, [get_total_charge(application_name, security_type) for application_name in application_names], bar_width, label=security_type, color=colors[i], alpha=opacity)
 
ax.set_xlabel('Application')
ax.set_ylabel('Charge [C]')
ax.set_title('Charge per Application')
fig.canvas.set_window_title('Charge per Application')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(application_names)
ax.legend()
 
fig.tight_layout()
fig.show()

# Number of Checkpoints

In [ ]:
security_types = ["None", "AES_Flash", "TruseZone_Flash"]

In [ ]:
cap_scale_per_app = 1e1
points_per_unit = 100
capacitor_sizes = np.arange(app_charge_scale*cap_scale_per_app,app_charge_scale*cap_scale_per_app*2, app_charge_scale*cap_scale_per_app/points_per_unit)
print(min(capacitor_sizes),max(capacitor_sizes))
print(base_charge_per_app["Kalman"])
print(get_security_charge("None", number_of_bytes_per_app["Kalman"]))
print(get_security_charge("AES_Flash", number_of_bytes_per_app["Kalman"]))

In [ ]:
opacity = 0.8
colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

figs = {}
axs = {}
for application_name in application_names:
    fig, ax = plt.subplots(figsize=(10,9))
    figs[application_name] = fig
    axs[application_name] = ax
    fig.canvas.set_window_title(application_name)
    for i,security_type in enumerate(security_types):
        ax.plot(capacitor_sizes, base_charge_per_app[application_name]/(capacitor_sizes - get_security_charge(security_type, number_of_bytes_per_app[application_name])), label=security_type, color=colors[i], alpha=opacity)
#     ax.set_yscale('log')
#     ax.set_xscale('log')
    ax.set_xlabel('Capacitor Size [C]')
    ax.set_ylabel('Number of Checkpoints')
    ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
    fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
    ax.legend()
    fig.tight_layout()
    fig.show()
    break

## Scales

In [ ]:
print("Cap Size:")
print(f"\tmin: {min(capacitor_sizes)}, max: {max(capacitor_sizes)}")

In [ ]:
print("Base Charge:")
for application_name in application_names:
    print(f"\t{application_name}: {base_charge_per_app[application_name]}")

In [ ]:
print("Security Charge:")
for security_type in security_types:
    print(f"\t{security_type} min: {get_security_charge(security_type, min(number_of_bytes_per_app.values()))},  max: {get_security_charge(security_type, max(number_of_bytes_per_app.values()))}")
# number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

In [ ]:
base_charge_per_app

In [ ]:
min(number_of_bytes_per_app.values())

In [ ]:
get_security_charge("None", min(number_of_bytes_per_app.values()))

In [ ]:
1544 + 2**10

In [ ]:
2**13

In [ ]:
620*16

## Plot stuff

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Capacitor Size [C]")
ax.set_ylabel("Number of Checkpoints")

In [ ]:
capacitor_sizes = range(1,101)
def get_number_of_checkpoints(application_number):
    return [application_charges[application_number]/(capacitor_size - get_security_charge("trustzone_flash", application_number_of_bytes[application_number])) for capacitor_size in capacitor_sizes]

In [ ]:
for application_number in range(5):
    ax.plot(capacitor_sizes, get_number_of_checkpoints(application_number), label=application_names[application_number])
ax.legend()
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.invert_xaxis()

In [ ]:
# fig2 = plt.figure(figsize=(10, 6))
# ax2 = fig2.add_subplot(111, projection='3d')

# ax2.set_xlabel("Capacitor Size [C]")
# ax2.set_ylabel("Number of Bytes")
# ax2.set_zlabel("Number of Checkpoints")

In [ ]:
# capacitor_sizes = range(1,1001)
# application_charge = 50
# number_of_bytes = range(1,1001)

In [ ]:
# def get_number_of_checkpoints(application_charge, capacitor_size, number_of_bytes, security_type="trustzone_flash"):
#     return application_charge/(capacitor_size - get_security_charge(security_type, number_of_bytes))

In [ ]:
# X, Y = np.meshgrid(capacitor_sizes, number_of_bytes)
# Z = np.zeros(shape=(1000,1000))
# for i,capacitor_size in enumerate(capacitor_sizes):
#     for j,number_of_byte in enumerate(number_of_bytes):
#         Z[i,j] = get_number_of_checkpoints(application_charge, capacitor_size, number_of_byte)
# #Z = np.array([np.array([get_number_of_checkpoints(capacitor_size,application_charge) for application_charge in application_charges]) for  capacitor_size in capacitor_sizes])
# Z.shape

In [ ]:
# surf = ax2.plot_surface(X, Y, Z)